In [1]:
import numpy as np
import dimod
import xtools as xt
import matplotlib.pyplot as plt
from openjij import SQASampler


In [2]:
from atm.flight.generator import ScenarioGenerator
from atm.separation import recat
from atm.flight.flight import DUMMY_FLIGHT_RECAT


In [84]:
cf = xt.Config(dict(
    dt=10,
    num=30,
    scenario=dict(
        interval=60,
        window=200,
        mode="mix",
        standard="recat"
    )
))


In [85]:
gen = ScenarioGenerator(cf.scenario)

In [44]:
sep = recat.TBS

In [45]:
def calc_assign_time_for_single_runway(vols, separation, indices):
    past_vol = DUMMY_FLIGHT_RECAT
    past_time = DUMMY_FLIGHT_RECAT.ready

    assigned_times = []
    for idx in indices:
        vol = vols[idx]
        sep_time = separation(past_vol, vol)
        time = np.max([vol.ready, past_time + sep_time])
        assigned_times.append(time)

        past_vol = vol
        past_time = time

    return assigned_times


In [109]:
def get_obj_multiple_runway(vols, separation, penalty):
    def func(xs):
        print(xs)
        times = np.array([
            calc_assign_time_for_single_runway(vols, separation, xs_)
            for xs_ in xs
        ])
        readys = np.array([
            [vols[x].ready for x in xs_]
            for xs_ in xs
        ])
        dues = np.array([
            [vols[x].due for x in xs_]
            for xs_ in xs
        ])

        print("times:", type(times), times)
        print("ready:", type(readys), readys)
        delays = times - readys
        delay = np.mean(delays)

        overtimes = dues - times
        is_overtimes = (overtimes < 0).astype(float)
        overtime_penalty = np.sum(is_overtimes) * penalty

        return delay + overtime_penalty
    return func

In [105]:
vols = gen(10)
idxs = [
    np.arange(len(vols) // 2) * 2,
    np.arange(len(vols) // 2) * 2 + 1
]
idxs

[array([0, 2, 4, 6, 8]), array([1, 3, 5, 7, 9])]

In [110]:
obj = get_obj_multiple_runway(vols, sep, 1000)
obj(idxs)


[array([0, 2, 4, 6, 8]), array([1, 3, 5, 7, 9])]
times: <class 'numpy.ndarray'> [[143 302 458 561 689]
 [214 310 516 614 696]]
ready: <class 'numpy.ndarray'> [[143 302 458 488 582]
 [214 310 463 528 662]]


35.3

In [111]:
class MultiSequenceMetropolis:

    def __init__(self, objective, num_item, num_queue, T=1., swap_remote=False, seed=None, on_notebook=False):
        self.obj = objective
        self.nx = num_item
        self.nq = num_queue
        self.T = T
        self._swap_remote = swap_remote
        self.rs = None
        self.seed(seed)
        self.on_notebook = on_notebook

        self.prob_swap_queue = 0.2

        self.ec = None
        self.best_x = None

    def seed(self, seed=None):
        self.rs = np.random.RandomState(seed)
        return seed

    def sample(self, x):
        x_ = x.copy()
        if self.prob_swap_queue <= self.rs.rand():
            print("change queue")
        print(x_)

    def step(self, xc):
        pass

    def accept_prob(self, ec, ep):
        de = ep - ec
        r = np.exp(-de / self.T)
        return np.min([1, r])

    def f(self, x):
        qxs = np.asarray([
            x[0, x[1,] == q]
            for q in range(self.nq)
        ])
        print(qxs)
        es = self.obj(qxs)
        # es = [self.obj(qx) for qx in qxs]
        print(qxs)
        print(es)

    def optimize(self, x0=None, num_iter=10):
        xc = x0 if x0 is not None else self.generate_base_x()
        self.ec = self.f(xc)
        pass

    @staticmethod
    def generate_base_x():
        return np.vstack([
            np.arange(len(vols)),
            np.zeros(len(vols))
        ]).astype(int)



algo = MultiSequenceMetropolis(get_obj_multiple_runway(vols, sep, 100), len(vols), 2)
xsample = np.vstack([
    np.arange(len(vols)),
    np.random.choice(np.arange(2), len(vols), replace=True)
]).astype(int)
print(xsample)
algo.f(xsample)
# algo.sample()

[[0 1 2 3 4 5 6 7 8 9]
 [0 0 1 0 0 0 0 0 0 0]]
[array([0, 1, 3, 4, 5, 6, 7, 8, 9]) array([2])]
[array([0, 1, 3, 4, 5, 6, 7, 8, 9]) array([2])]
times: <class 'numpy.ndarray'> [list([143, 217, 310, 458, 664, 746, 951, 1033, 1115]) list([302])]
ready: <class 'numpy.ndarray'> [list([143, 214, 310, 458, 463, 488, 528, 582, 662]) list([302])]


/var/folders/t7/nsdlqwpn6ggfjtwngwqx37x40000gn/T/ipykernel_41648/1936053741.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  qxs = np.asarray([
/var/folders/t7/nsdlqwpn6ggfjtwngwqx37x40000gn/T/ipykernel_41648/1341281347.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times = np.array([
/var/folders/t7/nsdlqwpn6ggfjtwngwqx37x40000gn/T/ipykernel_41648/1341281347.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecate

TypeError: unsupported operand type(s) for -: 'list' and 'list'